In [1]:
#!pip install neptune
#!pip install optuna
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from medmnist import DermaMNIST
import random
import neptune
import itertools
import optuna

In [2]:
torch.cuda.empty_cache()
class CNN(nn.Module):
    def __init__(self, dropout_rate):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=2, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)

        self.flattened_size = self.get_flattened_size((3, 64, 64))
        self.fc1 = nn.Linear(self.flattened_size, 64)
        self.fc2 = nn.Linear(64, 7)

    def get_flattened_size(self, input_shape):
        x = torch.zeros(1, *input_shape)
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        return x.view(1, -1).size(1)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Funkcja celu dla Optuny
def objective(trial):
    # Hiperparametry do tuningu
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
    dropout_rate = trial.suggest_uniform("dropout_rate", 0.2, 0.5)

    # Dane
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    train_dataset = DermaMNIST(root='./data', split="train", download=True, transform=transform, size=64)
    test_dataset = DermaMNIST(root='./data', split="test", download=True, transform=transform, size=64)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Model
    model = CNN(dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    # Trening
    for epoch in range(10):  # Mała liczba epok do szybkiego tuningu
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            target = target.long()  # Ensure target is of type torch.long
            if len(target.size()) > 1:
                target = target.squeeze()  # Remove extra dimensions
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

    # Ewaluacja
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            target = target.long()  # Ensure target is of type torch.long
            if len(target.size()) > 1:
                target = target.squeeze()  # Remove extra dimensions
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == target).sum().item()
            total += target.size(0)

    accuracy = correct / total
    return 1 - accuracy  # Minimalizujemy 1 - accuracy

# Główna funkcja
if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    study = optuna.create_study(direction="minimize")  # Minimalizacja (1 - accuracy)
    study.optimize(objective, n_trials=50)

    # Najlepsze hiperparametry
    print("Najlepsze hiperparametry: ", study.best_params)
    print("Najlepszy wynik: ", 1 - study.best_value)

[I 2025-01-24 11:11:52,852] A new study created in memory with name: no-name-41d3d76a-5bc2-4c2a-9e6b-04c3b42ddb1b
C:\Users\Kamil\AppData\Local\Temp\ipykernel_18492\2511716181.py:42: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
C:\Users\Kamil\AppData\Local\Temp\ipykernel_18492\2511716181.py:43: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform("dropout_rate", 0.2, 0.5)


Using downloaded and verified file: ./data\dermamnist_64.npz
Using downloaded and verified file: ./data\dermamnist_64.npz


[I 2025-01-24 11:12:04,381] Trial 0 finished with value: 0.33117206982543645 and parameters: {'batch_size': 256, 'learning_rate': 0.00014810540825968506, 'dropout_rate': 0.36348959137432685}. Best is trial 0 with value: 0.33117206982543645.


Using downloaded and verified file: ./data\dermamnist_64.npz
Using downloaded and verified file: ./data\dermamnist_64.npz


[I 2025-01-24 11:12:15,038] Trial 1 finished with value: 0.29975062344139647 and parameters: {'batch_size': 256, 'learning_rate': 0.0010050046393466406, 'dropout_rate': 0.2849458229595506}. Best is trial 1 with value: 0.29975062344139647.


...

Using downloaded and verified file: ./data\dermamnist_64.npz
Using downloaded and verified file: ./data\dermamnist_64.npz


[I 2025-01-24 11:26:27,205] Trial 48 finished with value: 0.2658354114713217 and parameters: {'batch_size': 32, 'learning_rate': 0.00039575463155468757, 'dropout_rate': 0.438100326353174}. Best is trial 42 with value: 0.22493765586034908.


Using downloaded and verified file: ./data\dermamnist_64.npz
Using downloaded and verified file: ./data\dermamnist_64.npz


[I 2025-01-24 11:26:51,628] Trial 49 finished with value: 0.2324189526184539 and parameters: {'batch_size': 32, 'learning_rate': 0.004910984463777503, 'dropout_rate': 0.34224204352183074}. Best is trial 42 with value: 0.22493765586034908.


Najlepsze hiperparametry:  {'batch_size': 32, 'learning_rate': 0.007081285830831507, 'dropout_rate': 0.44938018500900623}
Najlepszy wynik:  0.7750623441396509
